In [1]:
import deepchem as dc
from rdkit import Chem
from rdkit.Chem import MACCSkeys

Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (C:\Users\dietr004\AppData\Local\anaconda3\envs\spec2toxReadAcross\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading some Jax models, missing a dependency. jax requires jaxlib to be installed. See https://github.com/google/jax#installation for installation instructions.


In [3]:
tox21_tasks, tox21_datasets, tox21_transformers = dc.molnet.load_tox21(reload=False)

[11:53:58] WARNING: not removing hydrogen atom without neighbors
[11:54:04] WARNING: not removing hydrogen atom without neighbors


In [4]:
train_dataset, val_dataset, test_dataset = tox21_datasets
print(train_dataset.X.shape, train_dataset.y.shape)
print(val_dataset.X.shape, val_dataset.y.shape)
print(test_dataset.X.shape, test_dataset.y.shape)

(6264, 1024) (6264, 12)
(783, 1024) (783, 12)
(784, 1024) (784, 12)


In [44]:
smiles_train = train_dataset.ids
labels_train_ = train_dataset.y

In [45]:
smiles_val = val_dataset.ids
labels_val_ = val_dataset.y

In [46]:
mols_train = [Chem.MolFromSmiles(smi) for smi in smiles_train if Chem.MolFromSmiles(smi)]
maccs_train = [MACCSkeys.GenMACCSKeys(mol) for mol in mols_train]

[12:12:10] WARNING: not removing hydrogen atom without neighbors
[12:12:10] WARNING: not removing hydrogen atom without neighbors


In [47]:
mols_val = [Chem.MolFromSmiles(smi) for smi in smiles_val if Chem.MolFromSmiles(smi)]
maccs_val = [MACCSkeys.GenMACCSKeys(mol) for mol in mols_val]

---

In [104]:
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.metrics import balanced_accuracy_score

In [106]:
i = 0
labels_train = labels_train_[:,i]
labels_val = labels_val_[:,i]

In [107]:
smote = SMOTE(random_state=73)
maccs_train_resampled, labels_train_resampled = smote.fit_resample(maccs_train, labels_train)

In [108]:
rf = RandomForestClassifier()
rf.fit(maccs_train_resampled, labels_train_resampled)

RandomForestClassifier()

In [109]:
preds = rf.predict(maccs_val)

In [110]:
roc_auc_score(labels_val, preds)

0.6714353122855182

In [111]:
balanced_accuracy_score(labels_val, preds)

0.6714353122855181